In [13]:
! ../keras-retinanet/keras_retinanet/bin/train.py --weights data/resnet50_coco_best_v2.1.0.h5 \
--batch-size 2 --steps 500 --epochs 5 \
--snapshot-path data/snapshots_final/nms/optimized \
--random-transform \
--config config/config.ini \
csv data/mosaics/labeled_tiles/train_annotations_final.csv data/mosaics/labeled_tiles/classes.csv \
--val-annotations data/mosaics/labeled_tiles/val_annotations_final.csv

Using TensorFlow backend.
2020-05-09 02:56:29.085887: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-05-09 02:56:29.086965: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
usage: train.py [-h]
                [--snapshot SNAPSHOT | --imagenet-weights | --weights WEIGHTS | --no-weights]
                [--backbone BACKBONE] [--batch-size BATCH_SIZE] [--gpu GPU]
                [--multi-gpu MULTI_GPU] [--multi-gpu-force]
                [--initial-epoch INITIAL_EPOCH] [--epochs EPOCHS]
                [--steps STEPS] [--lr LR] [--snapshot-path SNAPSHOT_PATH]
                [--tensorboard-dir TENSORBOARD_DIR] [--no-snapshots]
                [--no-evaluation] [--freeze-backbone] [--random-transform]
                [--image-min-side IMAGE_MIN_SIDE]
                [--image-max-side IMAGE_MAX_SIDE] [--no-resize]
                [--config CONF

In [7]:
! ../keras-retinanet/keras_retinanet/bin/convert_model.py --config config/config.ini --nms-threshold 0.05 data/snapshots_final/nms/optimized/resnet50_csv_25.h5 data/snapshots_final/nms/optimized/final_model_nms05.h5

Using TensorFlow backend.
2020-05-09 00:26:12.097280: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-05-09 00:26:12.098374: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
2020-05-09 00:26:12.832462: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-09 00:26:12.840635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-09 00:26:12.840809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2020-05-09 00:26:12.840833: I tenso

In [1]:
! ../keras-retinanet/keras_retinanet/bin/evaluate.py csv data/mosaics/labeled_tiles/test_annotations_final.csv data/mosaics/labeled_tiles/classes.csv data/snapshots_final/nms/optimized/final_model_nms05.h5 

Using TensorFlow backend.
2020-05-09 00:28:53.031477: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-05-09 00:28:53.068142: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Loading model, this may take a second...
2020-05-09 00:28:55.094929: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-09 00:28:55.128142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-09 00:28:55.129916: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
Running network: N/A% (0 of 74) |        | Elapsed Time: 0:00:00 ETA:  --:--:--2020-05-09 00:29:02.235945: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-05-09 00:29:04.084167: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
Running network: 100% (74 of 74) |#######| Elapsed Time: 0:00:22 Time:  0:00:22
Parsing annotations: 100% (74 of 74) |###| Elapsed Time: 0:00:00 Time:  0:00:00
1594 instances of class albatross with average precision: 0.9779
Inference time for 74 images: 0.2148
mAP using the weighted average of precisions among classes: 0.9779
mAP: 0.9779


In [1]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet.models import retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import json
from random import shuffle

import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

Using TensorFlow backend.


In [2]:
model_path = 'data/snapshots_final/nms/optimized/final_model_nms05.h5'

print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')

print(model.summary())

labels_to_names = {0: 'albatross'}

data/snapshots_final/nms/optimized/final_model_nms05.h5
tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -9.708478 ,  -5.339663 ,   9.708478 ,   5.339663 ],
       [-14.8863325,  -8.187483 ,  14.8863325,   8.187483 ],
       [-23.731836 , -13.052509 ,  23.731836 ,  13.052509 ],
       [ -8.358542 ,  -6.2020383,   8.358542 ,   6.2020383],
       [-12.816432 ,  -9.509792 ,  12.816432 ,   9.509792 ],
       [-20.431993 , -15.160539 ,  20.431993 ,  15.160539 ],
       [ -7.2      ,  -7.2      ,   7.2      ,   7.2      ],
       [-11.04     , -11.04     ,  11.04     ,  11.04     ],
       [-17.6      , -17.6      ,  17.6      ,  17.6      ],
       [ -6.1990695,  -8.362545 ,   6.1990695,   8.362545 ],
       [ -9.50524  , -12.822569 ,   9.50524  ,  12.822569 ],
       [-15.153282 , -20.441776 ,  15.153282 ,  20.441776 ],
       [ -5.33993  ,  -9.707993 ,   5.33993  ,   9.707993 ],
       [ -8.187893 , -14.885589 ,   8.187893 ,  14.885589 ],
       [-13.053163 

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
image_dir = "data/mosaics/SteepleJason_NTiptoGully_transparent_mosaic_group1_500x500/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.png')):
            image_list.append(image_dir + filename)
print(len(image_list))

7128


In [9]:
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype(float)
    
    # initialize the list of picked indexes	
    pick = []
    
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        
        
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        # compute the ratio of overlap
        overlap = float(w * h) / area[idxs[:last]]
        
        # delete all indexes from the index list that are in the suppression list
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
    
    # return only the bounding boxes that were picked
    return boxes[pick].astype(int)

In [11]:
#non max suppression test 
visualize = True
min_score = 0.5

detections = {}

total_time = 0

count = 0

for image_path in image_list: 
    
    image = read_image_bgr(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    boxes = non_max_suppression_fast(boxes[0].astype(int), 0.1)
    
    for box, score, label in zip(boxes, scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list))

TypeError: only size-1 arrays can be converted to Python scalars

In [10]:
visualize = True
min_score = 0.5

detections = {}

total_time = 0

count = 0
detection_iterations = 10
shuffle(image_list)

for image_path in image_list: 
    if count > detection_iterations:
        break
    else: 
        count +=1
    
    image = read_image_bgr(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list))

IndexError: index 1 is out of bounds for axis 0 with size 1

In [29]:
min_score = 0.5

detections = {}

total_time = 0

for image_path in image_list:
    
    image = read_image_bgr(image_path)
          
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start
    
    boxes /= scale  
        
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []
        
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)    
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
            
print("Finished, time per image:", total_time/len(image_list))

Finished, time per image: 0.12015957506551218


In [30]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [31]:
with open('data/detections_final/nms/optimized/detections_steeplejason_ntipgully_nms05.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)